### Some functions you often use

In [ ]:
import torch

#device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print (device)

# tensor
X = torch.rand(1, 28, 28, device=device)
print(X)
print(X.shape)

a.tolist()  # returns the tensor as a list
a.item()   # Returns a value of this tensor as a standard Python number. Only works for tensor with one element.
a.unsqueeze(1)  # If you need add a new dimension in tensor, you can use squeeze function.

###  Load custome Dataset

In [ ]:
import os 
import pandas as pd
import torch
import numpy as np
import torchvision
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from PIL import Image
from torch.utils.data import DataLoader



class customDataset(Dataset):
    def __init__(self, csv_file, data_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.data_dir = data_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self, index):
        img_path = os.path.join(self.data_dir, self.annotations.iloc[index,1])
        image = Image.open(img_path)
        y_label = torch.tensor(int(self.annotations.iloc[index,2]))
        
        if self.transform:
            image = self.transform(image)
            
        return (image, y_label)
    

transform = transforms.Compose([
                                       transforms.Resize((224, 224)),
                                       transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])

train_set = customDataset(csv_file='./data/train_CapsuleEndos.csv', data_dir='./data/train_CapsuleEndos', \
                      transform=transform)   # 37191, 8 patients

test_set = customDataset(csv_file='./data/test_CapsuleEndos.csv', data_dir='./data/test_CapsuleEndos', \
                      transform=transform)    # 10197, 2 patients

# train_set, test_set = torch.utils.data.random_split(dataset, [40000, 7388])


### Load ResNet34 from torchvision

In [ ]:
from torch import nn
from torchvision.models import resnet18

model = resnet18()
model.fc = nn.Linear(512, 3, bias=True)
model.to('cuda')
model

### Load ResNet50

In [ ]:
from torch import nn
from torchvision.models import mobilenet_v3_small

model = mobilenet_v3_small()
model.classifier[3] = nn.Linear(1024, 3, bias=True)
model.to('cuda')
model

### Load EfficientNet b0

In [ ]:
from efficientnet_pytorch import EfficientNet

model = EfficientNet.from_pretrained("efficientnet-b0", num_classes=3).cuda()
model

### Initialization: Dataset, Loss, Optimizer and visualization

In [ ]:

from torch import nn
from torch import optim
from torch.utils.tensorboard import SummaryWriter

# train_loader = DataLoader(dataset=train_set, batch_size=8, shuffle=True)
init_lr = 0.1
criterion = nn. CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=init_lr)
# print(dir(optimizer))
# Set lr_scheduler

lambda1 = lambda epoch:  0.35 **epoch
scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda1)


writer = SummaryWriter('runs/CapsuleEndos/tensorboard')


### Check Accuracy

In [ ]:
# model.eval()



# def check_accuracy(loader, model):
#     num_correct = 0
#     num_samples = 0
#     model.eval()
    
#     with torch.no_grad():
#         for images, labels in loader:
#             images = images.to('cuda')
#             labels = labels.to('cuda')
#             results = model(images)
#             _, predictions = results.max(1)
#             num_correct += (predictions==labels).sum()
#             num_samples += predictions.size(0)
        
#         print(f'Got{num_correct}/{num_samples} with accuracy \
#         {float(num_correct)/float(num_samples)*100}%')
        
import numpy as np        
        
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    
    Conf_matrix = np.zeros((3, 3))
    
    model.eval()
    
    with torch.no_grad():
        for images, labels in loader:
            images = images.to('cuda')
            labels = labels.to('cuda')
            results = model(images)
            _, predictions = results.max(1)
#             print(predictions)
            if labels==0:
                if predictions==labels:
                    Conf_matrix[0][0] += 1
                elif predictions==1:
                    Conf_matrix[0][1] += 1
                elif predictions==2:
                    Conf_matrix[0][2] += 1
            elif labels==1:
                if predictions==labels:
                    Conf_matrix[1][1] += 1
                elif predictions==0:
                    Conf_matrix[1][0] += 1
                elif predictions==2:
                    Conf_matrix[1][2] += 1
            elif labels==2:
                if predictions==labels:
                    Conf_matrix[2][2] += 1
                elif predictions==0:
                    Conf_matrix[2][0] += 1
                elif predictions==1:
                    Conf_matrix[2][1] += 1
                    
            num_correct += (predictions==labels).sum()
            num_samples += predictions.size(0)
        
        print(f'Got{num_correct}/{num_samples} with accuracy \
        {float(num_correct)/float(num_samples)*100}%')
        np.set_printoptions(suppress=True)
        print(Conf_matrix)
        

###  Train, check, and visualization

In [ ]:

from torch import nn
from torch import optim
from torch.utils.tensorboard import SummaryWriter
import torchvision

epochs = 10
batch_size = 128
step = 0

train_loader = DataLoader(dataset=train_set, batch_size=batch_size, \
                                  shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=True)

        
for epoch in range(epochs):
    running_loss = 0.0
    for batch_index, (data, targets) in enumerate(train_loader, 0):

        # images, labels = dataiter.next()
#         print('The shape of images is :', data.shape)
        data = data.to('cuda')
    
        targets = targets.to('cuda')

        # forward + backward + optimize
        outputs = model(data)
        loss = criterion(outputs, targets)
#         losses.append(loss.item())
#         print(loss)

        # zero the parameter gradients
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        
        

        # calculate accuracy
        _, predictions = outputs.max(1) 
        num_correct = (predictions==targets).sum()
        training_acc = float(num_correct)/float(data.shape[0])
#         accuracies.append(training_acc)

        # plot things to tensorboard
#                 image_grid = torchvision.utils.make_grid(data)
#                 writer.add_image('Stomach_img', image_grid)
#                 writer.add_histogram('fc', model.fc.weight)
        writer.add_scalar('Training Loss', loss, global_step=step)
        writer.add_scalar('Training Accuracy', training_acc, global_step=step)


        step += 1
        print(f'Step: {step}')
        print('Training Accuracy', training_acc)
    
    scheduler.step()
    check_accuracy(test_loader, model)
    torch.save(model.state_dict(), 'model_effi-b0_weights.pth')
    model.train()
    
    
    
    

###  Save and Load Model

In [ ]:
# Saving and Loading Model Weights
model = models.vgg16(pretrained=True)
torch.save(model.state_dict(), 'model_weights.pth')

model = models.vgg16() 
model.load_state_dict(torch.load('model_weights.pth'))

'''
be sure to call model.eval() method before inferencing to set the 
dropout and batch normalization layers to evaluation mode. 
Failing to do this will yield inconsistent inference results.
''' 

model.eval()


# Saving and Loading Models with Shapes
torch.save(model, 'model.pth')

model = torch.load('model.pth')

